# Big data? 🤗 Datasets to the rescue!

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
# Menginstal pustaka yang diperlukan untuk memproses dataset, mengevaluasi model, dan menggunakan tokenizer
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
# Menginstal pustaka zstandard untuk membaca file dengan ekstensi .zst
!pip install zstandard

In [ ]:
# Mengimpor fungsi load_dataset dari pustaka datasets
from datasets import load_dataset

# Mendefinisikan file data PUBMED dari URL dalam format JSON terkompresi (.zst)
data_files = "https://huggingface.co/datasets/qualis2006/PUBMED_title_abstracts_2020_baseline/resolve/main/PUBMED_title_abstracts_2020_baseline.jsonl.zst"

# Memuat dataset PUBMED dalam mode "train" dari file JSON terkompresi
pubmed_dataset = load_dataset("json", data_files=data_files, split="train")
pubmed_dataset

Loading dataset shards:   0%|          | 0/49 [00:00<?, ?it/s]

Dataset({
    features: ['meta', 'text'],
    num_rows: 17722096
})

In [ ]:
# Menampilkan dataset PUBMED yang telah dimuat
pubmed_dataset[0]

{'meta': {'pmid': 1673585, 'language': 'eng'},
 'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was n

In [ ]:
# Menginstal pustaka psutil untuk memantau penggunaan RAM
!pip install psutil

In [ ]:
import psutil
# Menghitung dan mencetak jumlah RAM yang digunakan oleh proses Python (dalam MB)
# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used: 1220.04 MB


In [ ]:
# Mencetak jumlah file dalam dataset dan ukuran dataset dalam GB (cache file)
print(f"Number of files in dataset : {pubmed_dataset.dataset_size}")
size_gb = pubmed_dataset.dataset_size / (1024**3)
print(f"Dataset size (cache file) : {size_gb:.2f} GB")

Number of files in dataset : 24453015916
Dataset size (cache file) : 22.77 GB


In [ ]:
# Mengukur waktu iterasi melalui dataset dengan penghitungan batch
import timeit

code_snippet = """batch_size = 1000

for idx in range(0, len(pubmed_dataset), batch_size):
    _ = pubmed_dataset[idx:idx + batch_size]
"""

time = timeit.timeit(stmt=code_snippet, number=1, globals=globals())
print(
    f"Iterated over {len(pubmed_dataset)} examples (about {size_gb:.1f} GB) in "
    f"{time:.1f}s, i.e. {size_gb/time:.3f} GB/s"
)

Iterated over 17722096 examples (about 22.8 GB) in 324.1s, i.e. 0.070 GB/s


In [ ]:

# Memuat dataset dalam mode streaming untuk menghindari masalah memori
pubmed_dataset_streamed = load_dataset(
    "json", data_files=data_files, split="train", streaming=True
)

In [ ]:

# Mengambil elemen pertama dari dataset yang dimuat secara streaming
next(iter(pubmed_dataset_streamed))

{'meta': {'pmid': 1673585, 'language': 'eng'},
 'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was n

In [ ]:
# Mengimpor tokenizer DistilBERT dari Transformers
from transformers import AutoTokenizer
# Memuat tokenizer DistilBERT yang telah dilatih sebelumnya
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# Menerapkan tokenisasi pada dataset streaming
tokenized_dataset = pubmed_dataset_streamed.map(lambda x: tokenizer(x["text"]))

# Mengambil elemen pertama dari dataset yang telah ditokenisasi
next(iter(tokenized_dataset))

{'meta': {'pmid': 1673585, 'language': 'eng'},
 'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was n

In [ ]:

# Mengacak dataset streaming dengan buffer sebesar 10.000 elemen
shuffled_dataset = pubmed_dataset_streamed.shuffle(buffer_size=10_000, seed=42)
# Mengambil elemen pertama dari dataset yang telah diacak
next(iter(shuffled_dataset))

{'meta': {'pmid': 1675166, 'language': 'ita'},
 'text': '[Benzodiazepine withdrawal syndrome].\nBenzodiazepines (BDZ) are widely prescribed in clinical practice for many pathological conditions, because of their anxiolytic, sedative, myorelaxant and anticonvulsant properties. The effectiveness, specificity and rapidity of action, the few side effects and the virtual absence of toxicity, have contributed to the widespread use of these compounds. In the last decade, however, the attitude towards BDZ has greatly changed, due to growing awareness and concern about dependence liability, withdrawal phenomena, and long-term side effects. Withdrawal symptoms have been singled out and specified in the contest of a well-defined syndrome with foreseeable onset, duration and remission. Psychic and physical symptoms and disorders of sensory perception can be observed. These manifestations can be suppressed by resuming treatment. The symptomatic and developmental aspects of BDZ withdrawal syndrome a

In [ ]:
# Mengambil lima elemen pertama dari dataset streaming
dataset_head = pubmed_dataset_streamed.take(5)
list(dataset_head)

[{'meta': {'pmid': 1673585, 'language': 'eng'},
  'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was

In [ ]:
# Membuat dataset pelatihan dengan melewati 1.000 elemen pertama dari dataset yang diacak
train_dataset = shuffled_dataset.skip(1000)
# Membuat dataset validasi dengan mengambil 1.000 elemen pertama dari dataset yang diacak
validation_dataset = shuffled_dataset.take(1000)